Notebook for plotting Corona data from SSI 

Run SSI_get_data and ssh_get_private_udbydere_data first to get a dataset of the right format.

Data are read from subfolder : data
Date of current dataset is read from : data_date.dat 
Data from "private udbydere" is read from : data_private.dat

In [ ]:
#Limit dates to plot
N_days=60 # number of days to include
#N_days=150 # number of days to include
bad_cutoff=25000 # minimum number of tests to consider good
#bad_cutoff=100

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from datetime import date, timedelta
import pickle

In [ ]:
# Read date of last data-download

f = open("data_date.dat", 'rb')
date_str=pickle.load(f)
f.close()

print("Dataset is from: " + date_str)

In [ ]:
#define the file to read
datafolder=Path("data/")
datafile=datafolder / "Test_pos_over_time.csv"

In [ ]:
# Read datafile
# Skips last two lines (which does not convert to date) and converts index to date
# Notice handeling of danish format of the numbers (both decimal and thousands)
df=pd.read_csv(datafile,  sep=';', parse_dates=['Date'], index_col=['Date'],error_bad_lines=False, engine='python', skipfooter=2, decimal=',', thousands='.')


In [ ]:
# calculate some more numbers

# Positive emperical scaled by number of tests to power of 0.7  
# This scaling is based on results in 
# SSI "Ekspertrapport af d. 23. oktober 2020 Incidens og fremskrivning af COVID-19 tilfælde"
# https://www.ssi.dk/-/media/ssi-files/ekspertrapport-af-den-23-oktober-2020-incidens-og-fremskrivning-af-covid19-tilflde.pdf?la=da
def calcScaledNumber (row):
    if row.NotPrevPos > 0 :
        return row.NewPositive / (row.NotPrevPos**0.7) * 50000**0.7 / 50000 *100#Normalized positiv procent to 50000 tests
    else:
        return 0
    
df['ScaledNumber']=df.apply(lambda row: calcScaledNumber(row), axis=1)    

# Recalculate Positiv procent to get more decimals for plotting
def calcPosPct (row):
    if row.NotPrevPos > 0 :
        return row.NewPositive / row.NotPrevPos * 100
    else:
        return 0
df['PosPct']=df.apply(lambda row: calcPosPct(row), axis=1)


In [ ]:
#read data from "private udbydere"
#very experimental, based on the SSI_get_private_udbydere_data.ipynb notebook

f = open("data_private.dat", 'rb')
PU_data=pickle.load(f)
f.close()

In [ ]:
# for easy plot make a sub data frame with selected number of days 
df_sel=df[date.today()-timedelta(days=N_days):]

# and make index for "bad" datapoints
bad_idx=df_sel['NotPrevPos']<bad_cutoff

In [ ]:
# define a common title including date from file
title_str='SSI COVID-19 data, tilfælde opgjort på prøvetagningsdato \n' 
title_str += date_str

In [ ]:
axs=[None]*3 #define axs list as empty 3 entries
fig = plt.figure(figsize=(7, 10))
axs[0] = plt.subplot(311)
axs[1] = plt.subplot(312,sharex=axs[0])
axs[2] = plt.subplot(313,sharex=axs[0])


df_sel.plot(ax=axs[0],y='PosPct',title=title_str,label='NewPositive / NotPrevPosTested * 100',style='.');
df_sel[bad_idx].plot(ax=axs[0],y='PosPct',style='.',color='red',label='NewPositive / NotPrevPosTested * 100 (Tested<'+ str(bad_cutoff) + ')');
axs[0].set_ylabel("%");
axs[0].set_ylim(0,2)

df_sel.plot(ax=axs[1],y='NewPositive',style='.');
df_sel[bad_idx].plot(ax=axs[1],y='NewPositive',style='.',color='red',label='NewPositive (Tested<'+ str(bad_cutoff) + ')');

df_sel.plot(ax=axs[2],y='NotPrevPos',label='Tested (NotPrevPos)',style='.');
df_sel[bad_idx].plot(ax=axs[2],y='NotPrevPos',style='.',color='red',label='Tested<'+ str(bad_cutoff) + '');

In [ ]:
axs=[None]*4 #define axs list as empty 4 entries
fig = plt.figure(figsize=(7, 15))
axs[0] = plt.subplot(411)
axs[1] = plt.subplot(412,sharex=axs[0])
axs[2] = plt.subplot(413,sharex=axs[0])
axs[3] = plt.subplot(414,sharex=axs[0])


df_sel.plot(ax=axs[0],y='PosPct',title=title_str,label='NewPositive / NotPrevPosTested * 100',style='.');
df_sel[bad_idx].plot(ax=axs[0],y='PosPct',style='.',color='red',label='NewPositive / NotPrevPosTested * 100 (Tested<'+ str(bad_cutoff) + ')');
PU_data.plot(ax=axs[0],y='Positiv procent antigen',style='.',color='lime',label='Positiv / Tested * 100 (quick test: "antigen test")');
axs[0].set_ylabel("%");
axs[0].set_ylim(0,4.5)
axs[0].tick_params(which='both', bottom=True, top=True, left=True, right=True, direction='in')

df_sel.plot(ax=axs[1], y='ScaledNumber',label='NewPositive/NotPrevPosTested^0.7 * 50.000^0.7 / 50.000 *100',style='.');
df_sel[bad_idx].plot(ax=axs[1],y='ScaledNumber',style='.',color='red', label=' (Tested<'+ str(bad_cutoff) + ')');
axs[1].set_ylabel("Positiv Procent [Estimated for 50.000 tests]");
axs[1].tick_params(which='both', bottom=True, top=True, left=True, right=True, direction='in')
axs[1].set_ylim(0,4.5)

df_sel.plot(ax=axs[2],y='NewPositive',style='.');
df_sel[bad_idx].plot(ax=axs[2],y='NewPositive',style='.',color='red',label='NewPositive (Tested<'+ str(bad_cutoff) + ')');
PU_data.plot(ax=axs[2],y='Antal positiv antigen tests',style='.',color='lime',label='Positiv (quick test: "antigen test")');
axs[2].tick_params(which='both', bottom=True, top=True, left=True, right=True, direction='in')

df_sel.plot(ax=axs[3],y='NotPrevPos',label='Tested (NotPrevPos)',style='.');
df_sel[bad_idx].plot(ax=axs[3],y='NotPrevPos',style='.',color='red',label='Tested<'+ str(bad_cutoff) + '');
PU_data.plot(ax=axs[3],y='Antal antigen tests',style='.',color='lime',label='Tested (quick test: "antigen test")');
axs[3].tick_params(which='both', bottom=True, top=True, left=True, right=True, direction='in')

#save a pdf for printing
plt.savefig('All4.pdf')  